In [26]:
import pandas as pd
import tensorflow as tf

In [27]:
x_train = pd.read_csv('../data/tables/X_trainGBM.csv')
print(x_train.head())

  TargetName  TMA 27 R1 | 001 | Geometric Segment  \
0      ATP7B                            49.983944   
1       ERN1                            60.775932   
2      KRT74                            31.239965   
3    SPECC1L                            46.007948   
4      FOXJ1                            45.439949   

   TMA 27 R1 | 002 | Geometric Segment  TMA 27 R1 | 003 | Geometric Segment  \
0                            46.519148                            51.213741   
1                            49.501144                            42.021531   
2                            34.591161                            26.920043   
3                            47.115547                            43.991290   
4                            54.272339                            55.153259   

   TMA 27 R1 | 004 | Geometric Segment  TMA 27 R1 | 005 | Geometric Segment  \
0                            45.655397                            60.557470   
1                            50.591115          

In [28]:
last_row = x_train.index[-1]
x_train = x_train.drop(last_row)
print(x_train.tail())

      TargetName  TMA 27 R1 | 001 | Geometric Segment  \
18665      OR2H1                            31.807964   
18666    C1QTNF3                            44.303950   
18667      RAB42                            53.391940   
18668      PTPRJ                            45.439949   
18669       GPX7                            42.599952   

       TMA 27 R1 | 002 | Geometric Segment  \
18665                            29.819966   
18666                            36.380359   
18667                            53.079540   
18668                            56.657936   
18669                            41.151554   

       TMA 27 R1 | 003 | Geometric Segment  \
18665                            32.172734   
18666                            46.617636   
18667                            40.051771   
18668                            46.617636   
18669                            45.961049   

       TMA 27 R1 | 004 | Geometric Segment  \
18665                            25.912523   
18666      

In [29]:
x_train_transposed = x_train.transpose()
print(x_train_transposed.head())

                                         0          1          2      \
TargetName                               ATP7B       ERN1      KRT74   
TMA 27 R1 | 001 | Geometric Segment  49.983944  60.775932  31.239965   
TMA 27 R1 | 002 | Geometric Segment  46.519148  49.501144  34.591161   
TMA 27 R1 | 003 | Geometric Segment  51.213741  42.021531  26.920043   
TMA 27 R1 | 004 | Geometric Segment  45.655397  50.591115  44.421467   

                                         3          4          5      \
TargetName                             SPECC1L      FOXJ1     OR4K15   
TMA 27 R1 | 001 | Geometric Segment  46.007948  45.439949  58.503934   
TMA 27 R1 | 002 | Geometric Segment  47.115547  54.272339  58.447134   
TMA 27 R1 | 003 | Geometric Segment   43.99129  55.153259  51.870327   
TMA 27 R1 | 004 | Geometric Segment    33.3161  59.228623  53.058975   

                                         6          7          8      \
TargetName                                 PDC      MUC15   C1

In [30]:
x_train_transposed_np = x_train_transposed.to_numpy()
print(x_train_transposed_np)


[['ATP7B' 'ERN1' 'KRT74' ... 'RAB42' 'PTPRJ' 'GPX7']
 [49.98394374 60.77593159 31.23996484 ... 53.39193991 45.43994886
  42.59995205]
 [46.51914764 49.50114428 34.59116107 ... 53.07954026 56.65793623
  41.15155368]
 ...
 [55.78916555 62.55148865 48.46331553 ... 42.26451936 52.40800401
  32.68456164]
 [55.10729798 52.96026039 30.77420536 ... 34.35260134 73.71495703
  73.71495703]
 [46.88932654 48.12325618 40.71967831 ... 33.93306526 80.82239179
  61.07951746]]


In [42]:
import numpy as np
import keras

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = np.load('data/' + ID + '.npy')

            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [ ]:
# Parameters
params = {'dim': (32,32,32),
          'batch_size': 64,
          'n_classes': 6,
          'n_channels': 1,
          'shuffle': True}

# Datasets
partition = 272
labels = [0,1]

# Generators
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)

# Design model
model = Sequential()
[...] # Architecture
model.compile()

# Train model on dataset
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6)